In [1]:
import os
from pathlib import Path

import pandas as pd
import networkx as nx
import scipy.sparse as sp
import yaml

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

import mscproject.dataprep as dp
import mscproject.features as feat
import mscproject.simulate as sim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
conf_dict = yaml.safe_load(Path("config/conf.yaml").read_text())

# Load features
companies_features = pd.read_parquet(conf_dict["companies_features"])
persons_features = pd.read_parquet(conf_dict["persons_features"])
edges_features = pd.read_parquet(conf_dict["edges_features"])

In [4]:
all_edge_ids = edges_features["src"].append(edges_features["dst"]).unique()

/tmp/ipykernel_13315/2785072395.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_edge_ids = edges_features["src"].append(edges_features["dst"]).unique()


In [5]:
all_edge_ids

array(['2356236782051912119', '692314493058510508',
       '15829769449001705952', ..., '15233613116661558738',
       '7879326061664287605', '3003051962805999676'], dtype=object)

In [6]:
# select node ids that aren't in all_edge_ids
missing_companies = companies_features.query("id not in @all_edge_ids")
missing_persons = persons_features.query("id not in @all_edge_ids")

In [7]:
missing_companies

,id,component,isCompany,name,foundingDate,dissolutionDate,countryCode,companiesHouseID,openCorporatesID,openOwnershipRegisterID,...,closeness,clustering,pagerank,neighbour_count,neighbourhood_indegree,neighbourhood_outdegree,neighbourhood_closeness,neighbourhood_clustering,neighbourhood_pagerank,neighbourhood_neighbour_count


In [8]:
missing_persons

,id,component,isCompany,birthDate,name,nationality,is_anomalous,indegree,outdegree,closeness,clustering,pagerank,neighbour_count,neighbourhood_indegree,neighbourhood_outdegree,neighbourhood_closeness,neighbourhood_clustering,neighbourhood_pagerank,neighbourhood_neighbour_count


In [21]:
G = feat.make_graph(edges_features)
components = nx.weakly_connected_components(G)
node_id_cc_id_map ={
    node_id: component_id
    for component_id, component_nodes in enumerate(components)
    for node_id in component_nodes 
}
